In [1]:
import papermill as pm
import scrapbook as sb
import pandas as pd
from datetime import datetime, timedelta
import os

/Users/Wenhao/opt/anaconda3/envs/mfe-program/lib/python3.8/site-packages/papermill/iorw.py:50: FutureWarning: pyarrow.HadoopFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.
  from pyarrow import HadoopFileSystem


In [24]:
connection_string = 'sqlite:///data/data.db'
tokens = pd.read_sql('SELECT DISTINCT(token) FROM ohlc', connection_string).T.values.tolist()[0]

In [25]:
pd.read_sql('SELECT token, MIN(ts), MAX(ts) FROM ohlc GROUP BY token', connection_string)

,token,MIN(ts),MAX(ts)
0,AAVE,2021-11-01 00:00:00,2021-12-15 00:00:00
1,ADA,2021-11-01 00:00:00,2021-12-15 00:00:00
2,ATOM,2021-11-01 00:00:00,2021-12-15 00:00:00
3,AVAX,2021-11-01 00:00:00,2021-12-15 00:00:00
4,BTC,2021-11-01 00:00:00,2021-12-15 00:00:00
5,COMP,2021-11-01 00:00:00,2021-12-15 00:00:00
6,CRV,2021-11-01 00:00:00,2021-12-15 00:00:00
7,DOT,2021-11-01 00:00:00,2021-12-15 00:00:00
8,ETH,2021-11-01 00:00:00,2021-12-15 00:00:00
9,SOL,2021-11-01 00:00:00,2021-12-15 00:00:00


In [34]:
run_date = datetime(2021, 11, 29)
end_date = run_date + timedelta(days=14)

In [35]:
base_output_folder = os.path.expanduser('outputs')

if not os.path.exists(base_output_folder):
        os.makedirs(base_output_folder)

In [37]:
for token in tokens:
    print(f'running for {token} for date {run_date} to {end_date}', end='\r')
    res = pm.execute_notebook(
        'HW4_template.ipynb',
        f'{base_output_folder}/market_analysis_{token}_{run_date}.ipynb',
        parameters = {
            'from_date': f'{run_date}',
            'to_date': f'{end_date}',
            'token': token,
            'connection_string': connection_string
        }
    )

Executing:   0%|          | 0/13 [00:00<?, ?cell/s]

Executing:   0%|          | 0/13 [00:00<?, ?cell/s]

Executing:   0%|          | 0/13 [00:00<?, ?cell/s]

Executing:   0%|          | 0/13 [00:00<?, ?cell/s]

Executing:   0%|          | 0/13 [00:00<?, ?cell/s]

...

Executing:   0%|          | 0/13 [00:00<?, ?cell/s]

Executing:   0%|          | 0/13 [00:00<?, ?cell/s]

Executing:   0%|          | 0/13 [00:00<?, ?cell/s]

Executing:   0%|          | 0/13 [00:00<?, ?cell/s]

Executing:   0%|          | 0/13 [00:00<?, ?cell/s]

In [38]:
nbs = sb.read_notebooks('outputs')

In [39]:
pd.DataFrame([
    pd.Series({
        'token': scrap['token'].data,
        'from_date': scrap['from_date'].data,
        'to_date': scrap['to_date'].data,
        'return': scrap['prices'].data['return'].mean(),
        'volatility': scrap['prices'].data.volatility.mean(),
        'maxdrawdown': scrap['prices'].data.maxdrawdown.min(),
        'range': scrap['prices'].data.close[-1] - scrap['prices'].data.close[0]}) 
    for book, scrap in nbs.notebook_scraps.items()
])

,token,from_date,to_date,return,volatility,maxdrawdown,range
0,AAVE,2021-11-29 00:00:00,2021-12-13 00:00:00,-0.000735,0.212356,-0.378718,-58.6450
1,ADA,2021-11-29 00:00:00,2021-12-13 00:00:00,-0.000347,0.217477,-0.312827,-0.2445
2,ATOM,2021-11-29 00:00:00,2021-12-13 00:00:00,-0.000132,0.299425,-0.329570,-2.7560
3,AVAX,2021-11-29 00:00:00,2021-12-13 00:00:00,-0.000471,0.286536,-0.385790,-22.9300
4,BTC,2021-11-29 00:00:00,2021-12-13 00:00:00,-0.000298,0.154894,-0.206462,-6999.8400
5,COMP,2021-11-29 00:00:00,2021-12-13 00:00:00,-0.000978,0.224354,-0.377871,-88.2800
6,CRV,2021-11-29 00:00:00,2021-12-13 00:00:00,-0.000537,0.328177,-0.394862,-1.0362
7,DOT,2021-11-29 00:00:00,2021-12-13 00:00:00,-0.000357,0.245633,-0.350800,-6.0050
8,ETH,2021-11-29 00:00:00,2021-12-13 00:00:00,-0.000009,0.174104,-0.188101,-141.7100
9,SOL,2021-11-29 00:00:00,2021-12-13 00:00:00,-0.000279,0.242432,-0.317207,-26.6870
